In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import root
from matplotlib.colors import LinearSegmentedColormap
from scipy.optimize import fsolve
from scipy.ndimage import gaussian_filter1d
import glob
import os

In [ ]:
NI = N
NE = N

# Crear malla fina para el fondo
E_bg, I_bg = np.meshgrid(np.linspace(0, 1, 300), np.linspace(0, 1, 300))
# Definir el gradiente como interpolación lineal entre dos esquinas
gradient = -np.arctan(5*(E_bg - I_bg))
# Crear el mapa de color personalizado
cmap = LinearSegmentedColormap.from_list('strong_gradient', ['blue','blue', 'red', 'red'])
# Definimos la función de activación (sigmoide tipo tanh truncada)
def f(s):
    return np.where(s >= 0, np.tanh(s), 0)

# Parámetros del modelo
alpha = 0.1
w_EE = 1
w_EI = 0.5
w_IE = 1
w_II = 0.5
h_E = 0.001
h_I = 0.001

# Campo vectorial de las ecuaciones de Wilson-Cowan
def dE_dt(E, I):
    s_E = w_EE * E - w_EI * I + h_E
    return -alpha * E + (1 - E) * f(s_E)

def dI_dt(E, I):
    s_I = w_IE * E - w_II * I + h_I
    return -alpha * I + (1 - I) * f(s_I)

# Definir sistema de ecuaciones para encontrar el punto fijo (intersección de nulclinas)
def system(vars):
    E, I = vars
    eq1 = dE_dt(E, I)
    eq2 = dI_dt(E, I)
    return [eq1, eq2]


# Crear una malla de puntos en el espacio E-I
E_vals = np.linspace(0, 1, 20)
I_vals = np.linspace(0, 1, 20)
E, I = np.meshgrid(E_vals, I_vals)
dE = dE_dt(E, I)
dI = dI_dt(E, I)


# Usar fsolve para encontrar el punto fijo
initial_guess = [0.5, 0.5]
E_fp, I_fp = fsolve(system, initial_guess)

# Calcular nulclinas: puntos donde dE/dt = 0 y dI/dt = 0
E_nc = np.linspace(0, 1, 1000)
I_nc = np.linspace(0, 1, 1000)
EE, II = np.meshgrid(E_nc, I_nc)
nullcline_E = dE_dt(EE, II)
nullcline_I = dI_dt(EE, II)
# Recalcular las nulclinas como listas separadas sin usar tuplas
E_null_E = []
I_null_E = []
E_null_I = []
I_null_I = []

for e in E_nc:
    i_e = fsolve(lambda i: dE_dt(e, i), 0.00001)[0]
    if 0 <= i_e <= 1:
        E_null_E.append(e)
        I_null_E.append(i_e)

    i_i = fsolve(lambda i: dI_dt(e, i), 0.00001)[0]
    if 0 <= i_i <= 1:
        E_null_I.append(e)
        I_null_I.append(i_i)

E_null_E[-1]=E_null_E[-2]

#Evolución del sistema
NI_evol = []
NE_evol = []

for step in States:
    NI_active = sum(step[inhib_indices])
    NE_active = sum(step[excit_indices])
    NI_evol.append(NI_active/NI)
    NE_evol.append(NE_active/NE)

# Parámetros del modelo
WE = (w+w0)/2
WI = WE-w0
wEE = WE
wEI = WI
wIE = WE
wII = WI
hE = h
hI = h

#Campo vectorial y las nulclinas
E_field_vals = np.linspace(0, 1, 10)
I_field_vals = np.linspace(0, 1, 10)
E_field, I_field = np.meshgrid(E_field_vals, I_field_vals)

dE = dE_dt(E_field, I_field)
dI = dI_dt(E_field, I_field)

# Normalizar vectores
magnitude = np.sqrt(dE**2 + dI**2)
dE_norm = dE*3
dI_norm = dI*3

E_contour_vals = np.linspace(0, 1, 50)
I_contour_vals = np.linspace(0, 1, 50)
E_contour, I_contour = np.meshgrid(E_contour_vals, I_contour_vals)

dE_contour = dE_dt(E_contour, I_contour)
dI_contour = dI_dt(E_contour, I_contour)



In [ ]:

plt.figure(figsize=(8, 8))
# plt.imshow(gradient, extent=[0, 1, 0, 1], origin='lower', cmap=cmap, alpha=0.5, aspect='auto')
plt.streamplot(E_vals, I_vals, dE, dI, color="#0000004C", density=1.2)
# Nulclinas con plt.plot
plt.plot(E_null_E, I_null_E, 'b-', linewidth=2, alpha=0.8, label=r'$dE/dt = 0$')
plt.plot(E_null_I, I_null_I, 'r-', linewidth=2, alpha=0.8, label=r'$dI/dt = 0$')

plt.xlabel('E (excitación)')
plt.ylabel('I (inhibición)')
plt.plot(NE_evol, NI_evol, color=(0, 1, 0), label='Trayectoria')
plt.xlim((0,1))
plt.title('Espacio de fases del sistema de Wilson-Cowan determinista')
plt.grid(True)
# # plt.plot(E_fp, I_fp, 'ko', markersize=8, label='Punto fijo')
plt.legend(loc=2)
plt.show()


In [ ]:
def generar_raster():
    # ------------------------
    # Parámetros
    # ------------------------
    T = 0.1  # bin size en ms
    sigma_smooth = 5  # ms
    max_time = times[-1]  # solo hasta 1000 ms

    # ------------------------
    # Extraer spikes por neurona para eventplot
    # ------------------------
    num_steps, num_neurons = States.shape
    spike_times_per_neuron = [[] for _ in range(num_neurons)]

    for t_idx in range(num_steps):
        if times[t_idx] > max_time:
            break
        spiking_neurons = np.where(States[t_idx] == 1)[0]
        for neuron_id in spiking_neurons:
            spike_times_per_neuron[neuron_id].append(times[t_idx])

    # ------------------------
    # Calcular firing rate
    # ------------------------
    spike_times_all = np.concatenate([np.array(lst) for lst in spike_times_per_neuron if lst])
    t_min = times.min()
    t_max = max_time
    bins = np.arange(t_min, t_max + T, T)

    counts, _ = np.histogram(spike_times_all, bins=bins)
    T_sec = T / 1000.0
    firing_rate = counts / T_sec

    sigma_bins = sigma_smooth / T
    smoothed_firing_rate = gaussian_filter1d(firing_rate, sigma=sigma_bins)

    time_bins = bins[:-1] + T / 2

    # ------------------------
    # Plot superpuesto con twin axes
    # ------------------------
    fig, ax_raster = plt.subplots(figsize=(6, 6))

    # Raster plot (eje principal)
    ax_raster.eventplot(spike_times_per_neuron, colors='blue', alpha=0.01, linewidths=0.5)
    ax_raster.set_ylabel("Neuron index")
    ax_raster.set_xlabel("Time (ms)")
    ax_raster.set_xlim(t_min, t_max)
    ax_raster.set_ylim(0,800)

    # Segundo eje para firing rate
    ax_rate = ax_raster.twinx()
    ax_rate.plot(time_bins, smoothed_firing_rate, color="red", alpha=0.75, label="Mean firing rate", linewidth=2)
    ax_rate.set_ylabel("Firing rate (Hz)", color="black")
    ax_rate.tick_params(axis='y', labelcolor="black")
    ax_rate.set_xlim(t_min, t_max)
    ax_rate.set_ylim(0, np.max(smoothed_firing_rate))
    plt.tight_layout()
    plt.show()


In [ ]:
# Ruta a la carpeta
folder_path = "../Resultados/"

# Obtener lista de archivos .npz (ajusta la extensión si es necesario)
file_list = glob.glob(os.path.join(folder_path, "*.npz"))

# Loop
for file_path in file_list:
    print(f"Procesando archivo: {file_path}")

    # Cargar archivo
    data = np.load(file_path)
    States = data["States"]
    Rates = data["Rates"]
    Spikes = data["Spikes"]
    times = data["Times"]
    N = int(data["N"])
    w = float(data["w"])
    w0 = float(data["w0"])
    alpha = float(data["alpha"])
    h = float(data["h"])
    excit_indices = data["excit_indices"]
    inhib_indices = data["inhib_indices"]


    # Llamar a tu función
    print("Generando figura...")
    generar_raster()
